<a href="https://colab.research.google.com/github/Jahan08/Amber-tutorial/blob/main/Retrieve-Data-prostate-cancer-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 4.7 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0


In [3]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [16]:
target = new_client.target
target_query = target.search('LNCaP')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,LNCaP,18.0,False,CHEMBL612518,[],CELL-LINE,9606
1,[],Homo sapiens,LNCaP-Clone-FGC,13.0,False,CHEMBL2366172,[],CELL-LINE,9606
2,[],Homo sapiens,LNCaP C4-2,13.0,False,CHEMBL4296460,[],CELL-LINE,9606
3,[],Homo sapiens,LNCaP C4-2B,13.0,False,CHEMBL4296461,[],CELL-LINE,9606


In [17]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL612518'

In [18]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [19]:
df = pd.DataFrame.from_dict(res)
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,150220,[],CHEMBL708902,Inhibition of LNCaP human prostate carcinoma c...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,None
1,None,182085,[],CHEMBL708902,Inhibition of LNCaP human prostate carcinoma c...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,0.016
2,None,225608,[],CHEMBL708033,Compound was evaluated for cytotoxicity agains...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,nM,UO_0000065,None,200.0
3,None,233261,[],CHEMBL708033,Compound was evaluated for cytotoxicity agains...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,1.0
4,None,238309,[],CHEMBL708033,Compound was evaluated for cytotoxicity agains...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3756,None,24422693,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5049247,Antiproliferative activity against human LNCaP...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,200.0
3757,None,24456803,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5052974,Antiproliferative activity against human LNCaP...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,34.6
3758,None,24456804,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5052974,Antiproliferative activity against human LNCaP...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,34.6
3759,None,24456805,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5052974,Antiproliferative activity against human LNCaP...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,LNCaP,9606,None,None,IC50,uM,UO_0000065,None,34.6


In [8]:

# considering nM unit for MIC
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
len(df)

10163

In [10]:
df = df[df.canonical_smiles.notna()]
df = df[df.standard_value.notna()]

In [ ]:
# considering nM unit for MIC
df = df[(df['standard_units'] == '%')]
df = df[(df['relation'] == '=')]
len(df)

In [11]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i < 1000:
        STATUS.append("Yes") #active
        
    elif 10000 <= i <= 50000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

<ipython-input-11-20c913d0ecdd>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
0,None,70811,[],CHEMBL764519,Cytotoxicity of compound against PC-3 cell line,F,None,None,BAO_0000190,BAO_0000219,...,PC-3,9606,None,None,IC50,uM,UO_0000065,None,400.0,Yes
11,None,122315,[],CHEMBL857416,In vivo cytotoxic concentration of compound ag...,F,None,None,BAO_0000190,BAO_0000218,...,PC-3,9606,None,None,IC50,uM,UO_0000065,None,49.0,Yes
13,None,130771,[],CHEMBL857562,In vitro inhibitory concentration against huma...,F,None,None,BAO_0000190,BAO_0000219,...,PC-3,9606,None,None,IC50,nM,UO_0000065,None,5.1,Yes


In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 80:
        STATUS.append("Yes") #active
        
    elif i == 0:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

In [12]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

7375

In [13]:
df.to_csv('bioactivity_data_PC-3_IC50_nM.csv', index=False)
     